#Dataset preparation for FinALBERT Pre-training

In [ ]:
!nvidia-smi

Thu Oct 29 06:59:31 2020       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 455.23.05    Driver Version: 418.67       CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   52C    P0    28W / 250W |      0MiB / 16280MiB |      0%      Default |
|                               |                      |                 ERR! |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
import gc
gc.collect()

313

In [ ]:
!pip install pyarrow

In [ ]:
!pip install datasets

In [ ]:
# bookcorpus and ag_news dataset from huggingface
from datasets import load_dataset

dataset = load_dataset("bookcorpus", split='train[:10%]')

Reusing dataset bookcorpus (/root/.cache/huggingface/datasets/bookcorpus/plain_text/1.0.0/af844be26c089fb64810e9f2cd841954fd8bd596d6ddd26326e4c70e2b8c96fc)


In [ ]:
dataset_ag = load_dataset("ag_news", split='train')

Using custom data configuration default
Reusing dataset ag_news (/root/.cache/huggingface/datasets/ag_news/default/0.0.0/fb5c5e74a110037311ef5e904583ce9f8b9fbc1354290f97b4929f01b3f48b1a)


In [ ]:
dataset_ag

Dataset(features: {'text': Value(dtype='string', id=None), 'label': ClassLabel(num_classes=4, names=['World', 'Sports', 'Business', 'Sci/Tech'], names_file=None, id=None)}, num_rows: 120000)

In [ ]:
dataset_ag = dataset_ag.sort(column='label')

In [ ]:
dataset_ag['label']

In [ ]:
#filtering only business news from ag_news
dataset_ag = dataset_ag[60000:90000]

In [ ]:
import pandas as pd
import numpy as np
import re

## vocab file creation

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
#Reuters dataset
reuters = open("/content/drive/My Drive/ALBERTimplementation/data/reuters_docs.txt", "r")

In [ ]:
gc.collect()

216

##Preprocessing data

In [ ]:
def removeLinks(text):
    text = re.sub(r'http\S+', '', text, flags=re.MULTILINE)
    return text
headlines =  list(map(removeLinks, dataset_ag['text']))
documents = list(map(removeLinks, dataset['text']))

In [ ]:
gc.collect()

187

In [ ]:
def removeHTMLTags(text):
    text = re.sub(r'<.*?>', '', text, flags=re.MULTILINE)
    return text
headlines = list(map(removeHTMLTags, headlines))
documents = list(map(removeHTMLTags, documents))

In [ ]:
gc.collect()

155

In [ ]:
def removeWordsWithNumbers(text):
    return re.sub(r"\S*\d\S*", "", text).strip()
headlines = list(map(removeWordsWithNumbers, headlines))
documents = list(map(removeWordsWithNumbers, documents))

In [ ]:
gc.collect()

148

In [ ]:
def decontracted(phrase):
    # specific
    phrase = re.sub(r"won\'t", "will not", phrase)
    phrase = re.sub(r"can\'t", "can not", phrase)

    # general
    phrase = re.sub(r"n\'t", " not", phrase)
    phrase = re.sub(r"\'re", " are", phrase)
    phrase = re.sub(r"\'s", " is", phrase)
    phrase = re.sub(r"\'d", " would", phrase)
    phrase = re.sub(r"\'ll", " will", phrase)
    phrase = re.sub(r"\'t", " not", phrase)
    phrase = re.sub(r"\'ve", " have", phrase)
    phrase = re.sub(r"\'m", " am", phrase)
    return phrase
headlines = list(map(decontracted, headlines))
documents = list(map(decontracted, documents))

In [ ]:
def removePuctuations(text):
    return re.sub(r'[^A-Za-z0-9]+', ' ', text)
headlines = list(map(removePuctuations, headlines))
documents = list(map(removePuctuations, documents))

def removeWhiteSpaces(text):
    return text.strip()
headlines = list(map(removeWhiteSpaces, headlines))
documents = list(map(removeWhiteSpaces, documents))

In [ ]:
gc.collect()

0

In [ ]:
def convertToLowerCase(text):
    return text.lower()
headlines = list(map(convertToLowerCase, headlines))
documents = list(map(convertToLowerCase, documents))

In [ ]:
f = open("/content/drive/My Drive/ALBERTimplementation/data/kaggle/combineV2_book10per.txt", "x")
f = open("/content/drive/My Drive/ALBERTimplementation/data/kaggle/combineV2_book10per.txt", "w")

In [ ]:
import pandas as pd
df = pd.DataFrame({'Documents': reuters})
df.head()

,Documents
0,showers continued throughout the week in the b...
1,standard oil co and bp north america inc said ...
2,texas commerce bancshares inc s texas commerce...
3,bankamerica corp is not under pressure to act ...
4,the u s agriculture department reported the fa...


In [ ]:
for vocab in documents:
  f.write(vocab)
  f.write('\n')
  f.write('\n')
for vo in list(df):
  f.write(vo)
  f.write('\n')
  f.write('\n')
for v in headlines:
  f.write(v)
  f.write('\n')
  f.write('\n')  